# Tô màu ảnh đen trắng với SDXL + ControlNet Union Segment

Chuẩn ComfyUI logic — viết lại cho Colab!

In [1]:
!pip install -q diffusers transformers accelerate safetensors xformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 37.3 MB/s eta 0:00:00


In [31]:
import os

# Kiểm tra nếu file model.safetensors đã tồn tại
model_path ="RealVisXLV5_Lighting.safetensors"
if not os.path.exists(model_path):
    print("Model chưa có, đang tải về...")
    # Tải model nếu chưa có
    #!wget "https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors" -O model.safetensors
    !wget "https://civitai.com/api/download/models/798204?type=Model&format=SafeTensor&size=full&fp=fp16" -O RealVisXLV5_Lighting.safetensors
else:
    print("Model đã có sẵn.")
controlnet_path ="promax.safetensors"
if not os.path.exists(controlnet_path):
    print("controlnet_union_path chưa có, đang tải về...")
    # Tải model nếu chưa có
    !wget "https://huggingface.co/brad-twinkl/controlnet-union-sdxl-1.0-promax/resolve/main/diffusion_pytorch_model.safetensors" -O promax.safetensors
    #!wget "https://huggingface.co/xinsir/controlnet-union-sdxl-1.0/resolve/main/diffusion_pytorch_model.safetensors" -O controlnet_union_xl.safetensors
else:
    print("controlnet_union_path đã có sẵn.")


Model đã có sẵn.
controlnet_union_path chưa có, đang tải về...
--2025-04-15 18:32:13--  https://huggingface.co/brad-twinkl/controlnet-union-sdxl-1.0-promax/resolve/main/diffusion_pytorch_model.safetensors
Resolving huggingface.co (huggingface.co)... 18.164.174.118, 18.164.174.55, 18.164.174.17, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.118|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/08/46/08467bb0663f389ac88fb29639ece8d5b9a4a96503894bc9a5531cfec306ce58/9fae2e50cb431bfcbe05822b59ec2228df545ef27f711dea8949e9f4ed9f7cdc?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27diffusion_pytorch_model.safetensors%3B+filename%3D%22diffusion_pytorch_model.safetensors%22%3B&Expires=1744745533&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0NDc0NTUzM319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzA4LzQ2LzA4NDY3YmIwNjYzZjM4OWFjODhmYjI5NjM5ZWNlOGQ1

In [1]:
import torch
import gc
from PIL import Image
from diffusers import StableDiffusionXLControlNetPipeline, ControlNetModel

gc.collect()
torch.cuda.empty_cache()

model_path ="RealVisXLV5_Lighting.safetensors"
controlnet_path ="promax.safetensors"
# Load ControlNet Union Segment
print("Loading ControlNet Union...")
controlnet = ControlNetModel.from_single_file(
    controlnet_path,
    torch_dtype=torch.float16
)
controlnet.config.controlnet_conditioning_type = "segment"
print("✅ ControlNet loaded và set sang segment!")

gc.collect()
torch.cuda.empty_cache()

# Load SDXL Pipeline
pipe = StableDiffusionXLControlNetPipeline.from_single_file(
    model_path,
    controlnet=controlnet,
    torch_dtype=torch.float16,
    use_safetensors=True
)


pipe.to("cuda")
pipe.enable_xformers_memory_efficient_attention()
pipe.enable_sequential_cpu_offload()

gc.collect()
torch.cuda.empty_cache()



Loading ControlNet Union...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the model checkpoint were not used when initializing ControlNetModel: 
 ['control_add_embedding.linear_1.bias, control_add_embedding.linear_1.weight, control_add_embedding.linear_2.bias, control_add_embedding.linear_2.weight, spatial_ch_projs.bias, spatial_ch_projs.weight, task_embedding, transformer_layes.0.attn.in_proj_bias, transformer_layes.0.attn.in_proj_weight, transformer_layes.0.attn.out_proj.bias, transformer_layes.0.attn.out_proj.weight, transformer_la

✅ ControlNet loaded và set sang segment!


Fetching 17 files:   0%|          | 0/17 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
controlnet.config.controlnet_conditioning_type = "auto"
print("✅ ControlNet loaded và set sang segment!")

# Load SDXL Pipeline
pipe = StableDiffusionXLControlNetPipeline.from_single_file(
    model_path,
    controlnet=controlnet,
    torch_dtype=torch.float16,
    use_safetensors=True
)
# Đường dẫn Model
input_image_path = "2.jpeg"

# Load ảnh đen trắng
from PIL import Image

def resize_keep_aspect(image, max_res=1024):
    w, h = image.size
    if w > h:
        new_w = max_res
        new_h = int(h * max_res / w)
    else:
        new_h = max_res
        new_w = int(w * max_res / h)
    return image.resize((new_w, new_h), Image.LANCZOS)

input_image = Image.open(input_image_path).convert("RGB")
input_image = resize_keep_aspect(input_image, max_res=1024)


# Load ảnh đen trắng
input_image = Image.open(input_image_path).convert("RGB")
input_image = resize_keep_aspect(input_image, max_res=1024)
# Prompt từ workflow
prompt = "a realistic colored photograph of a Vietnamese couple, professtional photography, bright natural ambient light, realistic texture"
negative_prompt = "Negative(hands), text, error, cropped, , normal quality,  blurry"

print("🚀 Đang tô màu ảnh với ControlNet Union Segment...")
result = pipe(
    prompt=prompt,
    negative_prompt=negative_prompt,
    num_inference_steps=6,
    guidance_scale=1.5,
    image=input_image,
    denoise_strength = 1,
    controlnet_conditioning_scale=0.6
).images[0]

result.show()
result.save("/content/tocolor_output6.png")
print("✅ Đã lưu ảnh tại: /content/tocolor_output1.png")

✅ ControlNet loaded và set sang segment!


Fetching 17 files:   0%|          | 0/17 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

🚀 Đang tô màu ảnh với ControlNet Union Segment...


  0%|          | 0/6 [00:00<?, ?it/s]